In [203]:
import os
import pybullet_envs
import numpy as np
import matplotlib.pyplot as plt
import time
# import panda_gym # Not in requirements, required for panda envs, get latest from here: https://github.com/qgallouedec/panda-gym , don't use version from pypy 

from seagul.zoo3_utils import load_zoo_agent, ALGOS

path_to_zoo = "/home/asutay/Documents/rl-baselines3-zoo/" # Very hacky but this is what we do for now. use: git clone --recursive https://github.com/DLR-RM/rl-baselines3-zoo

#model_dir = os.path.abspath('') + "/keep_agents/bullet_works_rewards/" # Bullet envs with mdim and cdim rewards
#model_dir = os.path.abspath('') + "/keep_agents/default_post_01_001/" # Bullet with no extra reward, just policy refinement with ARS  
model_dir = os.path.abspath('') + "/keep_agents/train_all/"

In [204]:
# Iterate through save directory and load everything

model_dict = {}
env_dict = {}

for algo_file in os.scandir(model_dir):
    algo_dir = algo_file.path
    algo = algo_dir.split("/")[-1]
    model_dict[algo] = {}
    env_dict[algo] = {}
    for env_file in os.scandir(algo_dir):
        env_dir = env_file.path
        env_name = env_dir.split("/")[-1]
        model_dict[algo][env_name] = {}
        env_dict[algo][env_name] = {}
        env, original_model = load_zoo_agent(env_name, algo, zoo_path = path_to_zoo)
        env_dict[algo][env_name] = env
        model_dict[algo][env_name]['original'] = original_model
        for pkl_file in os.scandir(env_dir):
            post_name = pkl_file.path.split("/")[-1].split(".")[0]
            model_dict[algo][env_name][post_name] = ALGOS[algo].load(pkl_file.path, env=env, device='cpu')

/home/asutay/Documents/rl-baselines3-zoo//rl-trained-agents/ppo/AntBulletEnv-v0_1/AntBulletEnv-v0
Loading running average
with params: {'norm_obs': True, 'norm_reward': True}
/home/asutay/Documents/rl-baselines3-zoo//rl-trained-agents/ppo/ReacherBulletEnv-v0_1/ReacherBulletEnv-v0
Loading running average
with params: {'norm_obs': True, 'norm_reward': True}
/home/asutay/Documents/rl-baselines3-zoo//rl-trained-agents/ppo/HopperBulletEnv-v0_1/HopperBulletEnv-v0
Loading running average
with params: {'norm_obs': True, 'norm_reward': True}
/home/asutay/Documents/rl-baselines3-zoo//rl-trained-agents/ppo/HalfCheetahBulletEnv-v0_1/HalfCheetahBulletEnv-v0
Loading running average
with params: {'norm_obs': True, 'norm_reward': True}
/home/asutay/Documents/rl-baselines3-zoo//rl-trained-agents/ppo/Walker2DBulletEnv-v0_1/Walker2DBulletEnv-v0


/miniconda3/envs/policy_refinement/lib/python3.6/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


Loading running average
with params: {'norm_obs': True, 'norm_reward': True}
/home/asutay/Documents/rl-baselines3-zoo//rl-trained-agents/ddpg/AntBulletEnv-v0_1/AntBulletEnv-v0
/home/asutay/Documents/rl-baselines3-zoo//rl-trained-agents/ddpg/ReacherBulletEnv-v0_1/ReacherBulletEnv-v0
/home/asutay/Documents/rl-baselines3-zoo//rl-trained-agents/ddpg/HopperBulletEnv-v0_1/HopperBulletEnv-v0
/home/asutay/Documents/rl-baselines3-zoo//rl-trained-agents/ddpg/HalfCheetahBulletEnv-v0_1/HalfCheetahBulletEnv-v0
/home/asutay/Documents/rl-baselines3-zoo//rl-trained-agents/ddpg/Walker2DBulletEnv-v0_1/Walker2DBulletEnv-v0
/home/asutay/Documents/rl-baselines3-zoo//rl-trained-agents/tqc/AntBulletEnv-v0_1/AntBulletEnv-v0
/home/asutay/Documents/rl-baselines3-zoo//rl-trained-agents/tqc/ReacherBulletEnv-v0_1/ReacherBulletEnv-v0
/home/asutay/Documents/rl-baselines3-zoo//rl-trained-agents/tqc/HopperBulletEnv-v0_1/HopperBulletEnv-v0
/home/asutay/Documents/rl-baselines3-zoo//rl-trained-agents/tqc/HalfCheetahBulle

In [205]:
algo_list = list(model_dict.keys())
env_list = list(model_dict[algo_list[0]].keys())
post_list = list(model_dict[algo_list[0]][env_list[0]].keys())

print(algo_list) # Original Algorithm
print(env_list) # Environment

# Postprocessor, autogenerated names. "original" means the original agent from zoo without additional training, "postprocess_default" means just ARS with not extra reward.
# Any other name is some new reward functio 
print(post_list)

['ppo', 'ddpg', 'tqc', 'a2c', 'td3', 'sac']
['AntBulletEnv-v0', 'ReacherBulletEnv-v0', 'HopperBulletEnv-v0', 'HalfCheetahBulletEnv-v0', 'Walker2DBulletEnv-v0']
['original', 'postprocess_default']


In [44]:
env = env_dict['sac']['Walker2DBulletEnv-v0']
model = model_dict['sac']['Walker2DBulletEnv-v0'][post_list[0]] #change to post_list[1] for postprocess

In [206]:
#from seagul.zoo3_utils import do_rollout_stable
def do_rollout_mine(env, model, render=False, render_wait=0.01, seed=None):
    state_list = []
    act_list = []
    reward_list = []

    if seed:
        env.seed(seed)
    obs = env.reset()
    done = False

    while not done:
        # Add push or something else here 

        state_list.append(np.copy(obs))
        

        actions,_ = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(actions)
        
        # Or maybe here

    
        if render:
            env.render()
            time.sleep(render_wait)
        
        act_list.append(np.copy(actions))
        reward_list.append(reward)

    state_arr = np.stack(state_list).squeeze()
    act_arr = np.stack(act_list).squeeze()
    
    return state_arr, act_arr, reward_list, info


In [207]:
obs,act,rew,info = do_rollout_mine(env, model)

In [208]:
print(sum(rew))
print(info)
print(type(info))
type(info[0]['episode']['r'])

[2415.4558]
[{'TimeLimit.truncated': True, 'episode': {'r': 2415.458528, 'l': 1000, 't': 82.058881}, 'terminal_observation': array([-3.23349327e-01,  0.00000000e+00,  1.00000000e+00,  3.44059169e-01,
        0.00000000e+00, -2.47618160e-03,  0.00000000e+00, -7.08991587e-01,
        3.16140801e-01,  2.12449670e-01,  7.62875527e-02, -3.03854495e-01,
        2.74799526e-01, -1.20158382e-01,  6.17590904e-01, -2.45899245e-01,
       -3.75985354e-02,  1.52041882e-01,  1.00792265e+00,  8.04062234e-04,
        1.00000000e+00,  1.00000000e+00,  0.00000000e+00])}]
<class 'list'>


float

In [48]:
env = env_dict['sac']['Walker2DBulletEnv-v0']
model = model_dict['sac']['Walker2DBulletEnv-v0'][post_list[0]] #change to post_list[1] for postprocess

In [210]:
# Rollout Walker for no noise for n trials
env_name = env_list[4]
data_no_noise={}
row = 0

for k in enumerate(post_list):
    #for j in enumerate(env_list):
        for i in enumerate(algo_list):
        
            env = env_dict[i[1]][env_name]
            model = model_dict[i[1]][env_name][k[1]] # no refinement
            #model2 = model_dict[i[1]][j[1]][post_list[1]] # ARS policy refinement
            count = 1
            fail = 0
            rewards_list = []
            l_list = [] 
            n_trial = 300 #Trial number
            while count <= n_trial :
                obs,act,rew,info = do_rollout_mine(env, model)
                ep_rew = info[0]['episode']['r']
                ep_l = info[0]['episode']['l']

                rewards_list.append(ep_rew)
                l_list.append(ep_l)

                rewards_arr = np.stack(rewards_list).squeeze()
                l_arr = np.stack(l_list).squeeze()

                if info[0]['episode']['l']<1000:
                    fail+=1
                count+=1
            fail_percentage = 100*fail/n_trial

            reward_mean = np.mean(rewards_arr)
            reward_std = np.std(rewards_arr)
            l_mean = np.mean(l_arr)
#            if k[0] == 1:
#                data_no_noise[str(row)] = data_no_noise[str(row)]+[reward_mean,reward_std,fail_percentage,l_mean]
#            else:
#                data_no_noise[str(row)] = [j[1],i[1],reward_mean,reward_std,fail_percentage,l_mean]
            
            data_no_noise[str(row)] = [k[1],env_name,i[1],reward_mean,reward_std,fail_percentage,l_mean]
            print(row)
            row += 1


KeyboardInterrupt: 

In [91]:
#data_no_noise['0'] = data_no_noise['0']+[reward_mean,reward_std,fail_percentage,l_mean]
#print(data_no_noise)
import pandas as pd
#df=pd.DataFrame.from_dict(data_no_noise, orient='index', columns=['env', 'algo', 'post', 'r_mean', 'r_std', '%fail', 'l_mean'])
df_no_noise=pd.DataFrame.from_dict(data_no_noise, orient='index', columns=['env', 'algo', 'r_mean', 'r_std', '%fail', 'l_mean', 'r_mean_refined', 'r_std_refined', '%fail_refined', 'l_mean_refined'])
path = '~/Documents/GitHub/policy_refinement/Ty_files/Ty_csv/walker_no_noise.csv'
df_no_noise.to_csv(path,index = False)

df_no_noise

2.3333333333333335


,env,algo,r_mean,r_std,%fail,l_mean,r_mean_refined,r_std_refined,%fail_refined,l_mean_refined
0,AntBulletEnv-v0,ppo,2799.667353,0.0,0.0,1000.0,2952.524329,0.0,0.0,1000.0
1,AntBulletEnv-v0,tqc,3488.584792,0.0,0.0,1000.0,3638.116329,0.0,0.0,1000.0
2,AntBulletEnv-v0,td3,3341.984081,0.0,0.0,1000.0,3443.095194,0.0,0.0,1000.0
3,AntBulletEnv-v0,sac,3114.575600,0.0,0.0,1000.0,3246.050433,0.0,0.0,1000.0
4,ReacherBulletEnv-v0,ppo,25.185163,0.0,100.0,150.0,1.965402,0.0,100.0,150.0
5,ReacherBulletEnv-v0,tqc,8.856911,0.0,100.0,150.0,16.417330,0.0,100.0,150.0
6,ReacherBulletEnv-v0,td3,11.155703,0.0,100.0,150.0,9.163791,0.0,100.0,150.0
7,ReacherBulletEnv-v0,sac,8.638552,0.0,100.0,150.0,35.280945,0.0,100.0,150.0
8,HopperBulletEnv-v0,ppo,2616.276326,0.0,0.0,1000.0,2643.144418,0.0,0.0,1000.0
9,HopperBulletEnv-v0,tqc,2677.904975,0.0,0.0,1000.0,2661.941047,0.0,0.0,1000.0


In [202]:
for k in enumerate(post_list):
    for j in enumerate(env_list):
        for i in enumerate(algo_list):
        
            env = env_dict[i[1]][j[1]]
            model = model_dict[i[1]][j[1]][k[1]]
            a = env.observation_space.high
            print(k[1],i[1],j[1],a)
#print(.025*env.action_space.high)
#print(.025*(env.action_space.high-env.action_space.low)/2)
#np.random.normal(0,0.025,6)

original ppo AntBulletEnv-v0 [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf  1.]
original tqc AntBulletEnv-v0 [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf  1.]
original td3 AntBulletEnv-v0 [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf  1.]
original sac AntBulletEnv-v0 [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf  1.]
original ppo ReacherBulletEnv-v0 [inf inf inf inf inf inf inf inf inf  1.]
original tqc ReacherBulletEnv-v0 [inf inf inf inf inf inf inf inf inf  1.]
original td3 ReacherBulletEnv-v0 [inf inf inf inf inf inf inf inf inf  1.]
original sac ReacherBulletEnv-v0 [inf inf inf inf inf inf inf inf inf  1.]
original ppo HopperBulletEnv-v0 [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf  1.]
original t

In [195]:
# Define do_rollout with Gaussian noise added to the actions
def do_rollout_noise_action(env, model, render=False, render_wait=0.01, seed=None):
    state_list = []
    act_list = []
    reward_list = []

    if seed:
        env.seed(seed)
    obs = env.reset()
    done = False
    l = len(env.action_space.low)
    
    mean= 0 
    std = .025 # actions are normalized between -1 and 1 for all envs so using a static std for noise
    while not done:
        # Add push or something else here 

        state_list.append(np.copy(obs))
        

        actions,_ = model.predict(obs, deterministic=True)
        
        
        noise = np.reshape(np.random.normal(mean,std,l),(1,l))
        actions += noise #add gaussian noise to actions
        
        obs, reward, done, info = env.step(actions)
        
        # Or maybe here

    
        if render:
            env.render()
            time.sleep(render_wait)
        
        act_list.append(np.copy(actions))
        reward_list.append(reward)

    state_arr = np.stack(state_list).squeeze()
    act_arr = np.stack(act_list).squeeze()
    
    return state_arr, act_arr, reward_list, info

In [ ]:
# Define do_rollout with Gaussian noise on the observations
def do_rollout_noise_obs(env, model, render=False, render_wait=0.01, seed=None):
    state_list = []
    act_list = []
    reward_list = []

    if seed:
        env.seed(seed)
    obs = env.reset()
    done = False

    while not done:
        # Add push or something else here 

        state_list.append(np.copy(obs))
        

        actions,_ = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(actions)
        
        mean = 0
        std = 1 #to be determined per env
        obs += numpy.random.normal(mean,std,1) #sample one value from Gaussian and add to obs
        # Or maybe here

    
        if render:
            env.render()
            time.sleep(render_wait)
        
        act_list.append(np.copy(actions))
        reward_list.append(reward)

    state_arr = np.stack(state_list).squeeze()
    act_arr = np.stack(act_list).squeeze()
    
    return state_arr, act_arr, reward_list, info

In [196]:
# Run set number of trials and check fail/fall rate and mean rewards
data_noise_action={}
row = 0
env_name = env_list[4]
for k in enumerate(post_list):
    #for j in enumerate(env_list):
        for i in enumerate(algo_list):
        
            env = env_dict[i[1]][env_name]
            model = model_dict[i[1]][env_name][k[1]] # no refinement
            #model2 = model_dict[i[1]][j[1]][post_list[1]] # ARS policy refinement
            count = 1
            fail = 0
            rewards_list = []
            l_list = [] 
            n_trial = 100 #Trial number
            while count <= n_trial :
                obs,act,rew,info = do_rollout_noise_action(env, model)
                ep_rew = info[0]['episode']['r']
                ep_l = info[0]['episode']['l']

                rewards_list.append(ep_rew)
                l_list.append(ep_l)

                rewards_arr = np.stack(rewards_list).squeeze()
                l_arr = np.stack(l_list).squeeze()

                if info[0]['episode']['l']<1000:
                    fail+=1
                count+=1
            fail_percentage = 100*fail/n_trial

            reward_mean = np.mean(rewards_arr)
            reward_std = np.std(rewards_arr)
            l_mean = np.mean(l_arr)
#            if k[0] == 1:
#                data_no_noise[str(row)] = data_no_noise[str(row)]+[reward_mean,reward_std,fail_percentage,l_mean]
#            else:
#                data_no_noise[str(row)] = [j[1],i[1],reward_mean,reward_std,fail_percentage,l_mean]
            
            data_noise_action[str(row)] = [k[1],env_name,i[1],reward_mean,reward_std,fail_percentage,l_mean]
            row += 1

In [197]:
#data_no_noise['0'] = data_no_noise['0']+[reward_mean,reward_std,fail_percentage,l_mean]
#print(data_no_noise)
import pandas as pd
df=pd.DataFrame.from_dict(data_noise_action, orient='index', columns=[ 'post', 'env', 'algo', 'r_mean', 'r_std', '%fail', 'l_mean'])
#df=pd.DataFrame.from_dict(data_noise_action, orient='index', columns=['env', 'algo', 'r_mean', 'r_std', '%fail', 'l_mean', 'r_mean_refined', 'r_std_refined', '%fail_refined', 'l_mean_refined'])
path = '~/Documents/GitHub/policy_refinement/Ty_files/Ty_csv/walker_noise_2_5_pct.csv'
df.to_csv(path,index = False)

df

,post,env,algo,r_mean,r_std,%fail,l_mean
0,original,Walker2DBulletEnv-v0,ppo,2105.509806,15.345420,0.0,1000.00
1,original,Walker2DBulletEnv-v0,tqc,2517.398869,567.820513,7.0,944.94
2,original,Walker2DBulletEnv-v0,td3,2228.219013,111.098149,12.0,994.84
3,original,Walker2DBulletEnv-v0,sac,2264.467164,227.833899,3.0,988.48
4,postprocess_default,Walker2DBulletEnv-v0,ppo,2219.476561,273.200808,3.0,980.99
5,postprocess_default,Walker2DBulletEnv-v0,tqc,2744.322199,7.325113,0.0,1000.00
6,postprocess_default,Walker2DBulletEnv-v0,td3,2444.182995,7.772026,6.0,999.85
7,postprocess_default,Walker2DBulletEnv-v0,sac,2401.658417,13.947507,0.0,1000.00


In [199]:
# Load CSV from path and format it to merge mean and std 
# The column names need to be changed according to CSV
path_csv = '~/Documents/GitHub/policy_refinement/Ty_files/Ty_csv/walker_noise_2_5_pct.csv'
# read csv at path and change the data type of specific columns for rounding
df = pd.read_csv(path_csv,index_col=False, dtype =  {'post':str, 'env':str, 'algo':str, 'r_mean':np.double, 
                                    'r_std':np.double, '%fail':np.double, 'l_mean':np.double} )
#df_test = (df.groupby(df.columns[3], axis=1).apply(lambda x: x.astype(str).apply('±'.join, 1)))
df_test = df
df_test=df_test.round({'r_mean':2,'r_std':2,'%fail':2,'l_mean':2})

#merge mean and std with plusminus from this point on the column is type str
df_test['r_mean'] = df_test[['r_mean', 'r_std']].astype(str).agg('±'.join, axis=1) 

#get rid of the std column
df_test = df_test.drop('r_std',axis=1)
df_test




,post,env,algo,r_mean,%fail,l_mean
0,original,Walker2DBulletEnv-v0,ppo,2105.51±15.35,0.0,1000.00
1,original,Walker2DBulletEnv-v0,tqc,2517.4±567.82,7.0,944.94
2,original,Walker2DBulletEnv-v0,td3,2228.22±111.1,12.0,994.84
3,original,Walker2DBulletEnv-v0,sac,2264.47±227.83,3.0,988.48
4,postprocess_default,Walker2DBulletEnv-v0,ppo,2219.48±273.2,3.0,980.99
5,postprocess_default,Walker2DBulletEnv-v0,tqc,2744.32±7.33,0.0,1000.00
6,postprocess_default,Walker2DBulletEnv-v0,td3,2444.18±7.77,6.0,999.85
7,postprocess_default,Walker2DBulletEnv-v0,sac,2401.66±13.95,0.0,1000.00


In [200]:
# Prints the data from in Latex format
print("\\usepackage{booktabs}\n"+ df_test.to_latex(index=False))

\usepackage{booktabs}
\begin{tabular}{llllrr}
\toprule
                post &                   env & algo &          r\_mean &  \%fail &   l\_mean \\
\midrule
            original &  Walker2DBulletEnv-v0 &  ppo &   2105.51±15.35 &    0.0 &  1000.00 \\
            original &  Walker2DBulletEnv-v0 &  tqc &   2517.4±567.82 &    7.0 &   944.94 \\
            original &  Walker2DBulletEnv-v0 &  td3 &   2228.22±111.1 &   12.0 &   994.84 \\
            original &  Walker2DBulletEnv-v0 &  sac &  2264.47±227.83 &    3.0 &   988.48 \\
 postprocess\_default &  Walker2DBulletEnv-v0 &  ppo &   2219.48±273.2 &    3.0 &   980.99 \\
 postprocess\_default &  Walker2DBulletEnv-v0 &  tqc &    2744.32±7.33 &    0.0 &  1000.00 \\
 postprocess\_default &  Walker2DBulletEnv-v0 &  td3 &    2444.18±7.77 &    6.0 &   999.85 \\
 postprocess\_default &  Walker2DBulletEnv-v0 &  sac &   2401.66±13.95 &    0.0 &  1000.00 \\
\bottomrule
\end{tabular}

